In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
from sklearn.metrics import roc_curve, auc, average_precision_score, precision_recall_curve
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import tensorflow as tf  # Make sure to import TensorFlow for the squash function

# Define the path and folder name
path = "/content/drive/MyDrive/Watashara_Projects/NIA/"

# Load your test data
Xt = X_ind  # Ensure X_ind is defined in your environment
yt = y_ind  # Ensure y_ind is defined in your environment

# Reshape Xt to the appropriate shape for the model
[sample_num, input_dim] = np.shape(Xt)
Xt = Xt.reshape(-1, 1, input_dim, 1)  # Assign the reshaped array back to Xt

sepscores = []
ytest = np.ones((1, 2)) * 0.5
yscore = np.ones((1, 2)) * 0.5

for i in range(5):
    loaded_model = load_model(
        path + f'Results/{fld_name}/capsule_network_model_fold_{i + 1}.h5',
        custom_objects={'squash': squash}  # Register the custom function here
    )
    print("Loaded model from disk")

    # Evaluate loaded model on test data
    y_score = loaded_model.predict(Xt)
    y_class = categorical_probas_to_classes(y_score)

    y_test = to_categorical(yt)
    ytest = np.vstack((ytest, y_test))
    yscore = np.vstack((yscore, y_score))

    acc, precision, npv, sensitivity, specificity, mcc, f1 = calculate_performance(len(y_class), y_class, yt)

    fpr, tpr, _ = roc_curve(y_test[:, 1], y_score[:, 1])
    roc_auc = auc(fpr, tpr)  # Calculate ROC AUC

    # Calculate AUPR
    aupr = average_precision_score(y_test[:, 1], y_score[:, 1])

    # Calculate Precision-Recall AUC
    precision_vals, recall, _ = precision_recall_curve(y_test[:, 1], y_score[:, 1])
    pr_auc = auc(recall, precision_vals)

    sepscores.append([acc, sensitivity, specificity, mcc, f1, roc_auc, pr_auc])
    print(f'Fold {i + 1}: acc={acc:.6f}, sensitivity={sensitivity:.6f}, specificity={specificity:.6f}, '
          f'mcc={mcc:.6f}, f1={f1:.6f}, roc_auc={roc_auc:.6f}, pr_auc={pr_auc:.6f}')

# Prepare results for saving
row = ytest.shape[0]
ytest = ytest[np.array(range(1, row)), :]
ytest_sum = pd.DataFrame(data=ytest)
ytest_sum.to_csv(path + f'Results/{fld_name}/Caps_ytest_test.csv')

yscore_ = yscore[np.array(range(1, row)), :]
yscore_sum = pd.DataFrame(data=yscore_)
yscore_sum.to_csv(path + f'Results/{fld_name}/Caps_yscore_test.csv')

# Calculate mean results
scores = np.array(sepscores)
result1 = np.mean(scores, axis=0)
H1 = result1.tolist()
sepscores.append(H1)
result = sepscores

data_csv = pd.DataFrame(data=result)
data_csv.to_csv(path + f'Results/{fld_name}/Caps_allfeat_test_results.csv')


Loaded model from disk
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step
Fold 1: acc=0.936416, sensitivity=0.957447, specificity=0.911392, mcc=0.872016, f1=0.942408, roc_auc=0.966402, pr_auc=0.960335


Loaded model from disk
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step
Fold 2: acc=0.959538, sensitivity=0.946809, specificity=0.974684, mcc=0.919267, f1=0.962162, roc_auc=0.971115, pr_auc=0.960232


Loaded model from disk
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 116ms/step
Fold 3: acc=0.965318, sensitivity=0.978723, specificity=0.949367, mcc=0.930222, f1=0.968421, roc_auc=0.975761, pr_auc=0.971071


Loaded model from disk
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 116ms/step
Fold 4: acc=0.942197, sensitivity=0.936170, specificity=0.949367, mcc=0.883991, f1=0.946237, roc_auc=0.973270, pr_auc=0.962334


Loaded model from disk
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step
Fold 5: acc=0.971098, sensitivity=0.978723, specificity=0.962025, mcc=0.941764, f1=0.973545, roc_auc=0.976569, pr_auc=0.971659
